In [1]:
CSV_FILE = "./Resources/LomaLinda.csv"
classes_dict = {0:'Diorite', 1:'Gabbro', 2:'Granite', 3:'Granodiorite'}
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


#### 1. LOAD DATA

In [2]:
import numpy as np
data = np.loadtxt(CSV_FILE, delimiter=",")

#data = data[data[:,-1] != 0]
#data = data[data[:,-1] != 1]
#data = data[data[:,-1] != 2]
#data = data[data[:,-1] != 3]

x = data[:, :-1]
y = data[:, -1]
print('x shape:', x.shape, 'y shape:', y.shape)

classes, indices, counts = np.unique(y, return_counts=True, return_index=True)

print('Classes:', classes)
print('Counts:', counts)
print('Indices:', indices)
print('Classes dictionary:', classes_dict)

x shape: (283, 16) y shape: (283,)
Classes: [0. 1. 2. 3.]
Counts: [78 65 70 70]
Indices: [140 218  70   0]
Classes dictionary: {0: 'Diorite', 1: 'Gabbro', 2: 'Granite', 3: 'Granodiorite'}


#### 2. SPLIT DATA

In [3]:
# Val = 20%, test = 16%, train = 64%
from sklearn.model_selection import train_test_split as splitter
xtt, xval, ytt, yval = splitter(x,y,train_size=0.8,random_state=42)

xtrain, xtest, ytrain, ytest = splitter(xtt,ytt,train_size=0.8,random_state=42)

In [4]:
def get_uniques(arr, name):
    print('<<', name, '>>')
    classes, indices, counts = np.unique(arr, return_counts=True, return_index=True)
    print('Classes:', classes)
    print('Counts:', counts)
    print('Indices:', indices)

In [5]:
get_uniques(ytrain, 'Train')
get_uniques(ytest, 'Test')
get_uniques(yval, 'Val')

<< Train >>
Classes: [0. 1. 2. 3.]
Counts: [50 40 45 45]
Indices: [4 0 2 1]
<< Test >>
Classes: [0. 1. 2. 3.]
Counts: [11 12 12 11]
Indices: [2 3 1 0]
<< Val >>
Classes: [0. 1. 2. 3.]
Counts: [17 13 13 14]
Indices: [3 1 2 0]


#### 3. NORMALIZE DATA

In [6]:
from sklearn.preprocessing import MinMaxScaler
normalizer = MinMaxScaler()
xtrainNorm = normalizer.fit_transform(xtrain)
xtestNorm = normalizer.fit_transform(xtest)

#### 4. TRAINING MODEL

In [7]:
#   Logistic Regression
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression(solver='lbfgs')
logReg.fit(xtrainNorm, ytrain)

LogisticRegression()

In [8]:
#   KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(xtrainNorm, ytrain)

KNeighborsClassifier(n_neighbors=3)

In [10]:
# tf Graph Input
xtr = tf.placeholder("float", [180, 16])
xte = tf.placeholder("float", [16])

var = tf.get_variable('weights', dtype=tf.float32, shape=(180,16))
val = xtr + var
out = tf.identity(val, name='out')

# Nearest Neighbor calculation using L1 Distance
# Calculate L1 Distance
distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)
# Prediction: Get min distance index (Nearest neighbor)
pred = tf.arg_min(distance, 0)

accuracy = 0.

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # loop over test data
    for i in range(len(xtestNorm)):
        # Get nearest neighbor
        nn_index = sess.run(pred, feed_dict={xtr: xtrainNorm, xte: xtestNorm[i, :]})
        # Get nearest neighbor class label and compare it to its true label
        print("Test", i, "Prediction:", np.argmax(ytrain[nn_index]), \
            "True Class:", np.argmax(ytest[i]))
        # Calculate accuracy
        if np.argmax(ytrain[nn_index]) == np.argmax(ytest[i]):
            accuracy += 1./len(xtestNorm)
    print("Accuracy : ",accuracy)
    
    # Convert the model.
    converter = tf.lite.TFLiteConverter.from_session(sess, [xtr], [out])
    tflite_model = converter.convert()

    # Save the model.
    with open('model.tflite', 'wb') as f:
        f.write(tflite_model)

Test 0 Prediction: 0 True Class: 0
Test 1 Prediction: 0 True Class: 0
Test 2 Prediction: 0 True Class: 0
Test 3 Prediction: 0 True Class: 0
Test 4 Prediction: 0 True Class: 0
Test 5 Prediction: 0 True Class: 0
Test 6 Prediction: 0 True Class: 0
Test 7 Prediction: 0 True Class: 0
Test 8 Prediction: 0 True Class: 0
Test 9 Prediction: 0 True Class: 0
Test 10 Prediction: 0 True Class: 0
Test 11 Prediction: 0 True Class: 0
Test 12 Prediction: 0 True Class: 0
Test 13 Prediction: 0 True Class: 0
Test 14 Prediction: 0 True Class: 0
Test 15 Prediction: 0 True Class: 0
Test 16 Prediction: 0 True Class: 0
Test 17 Prediction: 0 True Class: 0
Test 18 Prediction: 0 True Class: 0
Test 19 Prediction: 0 True Class: 0
Test 20 Prediction: 0 True Class: 0
Test 21 Prediction: 0 True Class: 0
Test 22 Prediction: 0 True Class: 0
Test 23 Prediction: 0 True Class: 0
Test 24 Prediction: 0 True Class: 0
Test 25 Prediction: 0 True Class: 0
Test 26 Prediction: 0 True Class: 0
Test 27 Prediction: 0 True Class: 0
Te

ConverterError: See console for info.
/bin/sh: toco_from_protos: command not found




#### 5. TESTING MODEL

In [ ]:
from sklearn import metrics

In [ ]:
# Logistic Regression
print("<<MODELS REPORT LOG. REGRESSION>>")
print("Logistic Regression score:",logReg.score(xtestNorm, ytest))

predictions = logReg.predict(xtestNorm)
print("Logistic Regression Model accuracy:", metrics.accuracy_score(ytest, predictions))
print(metrics.classification_report(ytest,predictions))

In [ ]:
# KNN
print("<<MODEL REPORT KNN>>")
print("KNN score:",knn.score(xtestNorm, ytest))

predictions = knn.predict(xtestNorm)
print("KNN Model accuracy:", metrics.accuracy_score(ytest, predictions))
print(metrics.classification_report(ytest,predictions))

In [ ]:
import pandas as pd

def get_evaluation_dataframe(model, xdata, ydata, classes_dict):
    real = []; predicted = []; accuracy = []
    
    classes, indices, counts = np.unique(ydata, return_counts=True, return_index=True)
    
    well_predicted = {key:0 for key in classes}
    #well_predicted.fromkeys(classes, 0)
    
    probabilities = model.predict_proba(x)
    
    for (features, lbl, probs) in zip(xdata, ydata, probabilities):
        r = int(lbl)
        real.append(classes_dict[r])
        p = int(model.predict([features]))
        predicted.append(classes_dict[p])
        accuracy.append(max(probs))
        if(r == p):
            well_predicted[r] += 1
    
    list_of_tuples = list(zip(real, predicted, accuracy))
    
    i = 0
    for cl in classes:
        idx = int(cl)
        print(classes_dict[idx], well_predicted[idx], counts[i])
        i+=1
    
    df = pd.DataFrame(list_of_tuples, columns = ['Real class', 'Predicted class', 'Accuracy'])
    
    return df

In [ ]:
get_evaluation_dataframe(logReg, xval, yval, classes_dict)

In [ ]:
get_evaluation_dataframe(knn, xval, yval, classes_dict)